In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random

# Step 1: Prepare the dataset
# You need to provide a list of comedy routine sentences as your training data.
# For demonstration purposes, we'll use a small example dataset.

training_data5 = open('/content/AJ_TP_text_05.txt', 'r').read().splitlines()
training_data2 = open('/content/AJ_TP_text_02.txt', 'r').read().splitlines()
training_data3 = open('/content/AJ_TP_text_03.txt', 'r').read().splitlines()
training_data4 = open('/content/AJ_TP_text_04.txt', 'r').read().splitlines()
training_data6 = open('/content/AJ_TP_text_06.txt', 'r').read().splitlines()
training_data7 = open('/content/AJ_TP_text_07.txt', 'r').read().splitlines()
training_data = open('/content/AJ_TP_text_01.txt', 'r').read().splitlines()
for i in training_data2 :
    training_data.append(i)

for i in training_data3 :
    training_data.append(i)

for i in training_data4 :
    training_data.append(i)

for i in training_data5 :
    training_data.append(i)

for i in training_data6 :
    training_data.append(i)
for i in training_data7 :
    training_data.append(i)
# Step 2: Tokenize the data
# We'll convert the text into a list of words (tokens).

def tokenize(text):
    return text.split()

tokenized_data = [tokenize(sentence.lower()) for sentence in training_data]

# Step 3: Create word-to-index and index-to-word dictionaries

word_to_index = {}
index_to_word = {}
for sentence in tokenized_data:
    for word in sentence:
        if word not in word_to_index:
            index = len(word_to_index)
            word_to_index[word] = index
            index_to_word[index] = word

# Add special tokens for padding and end-of-sentence (EOS)
pad_token = '<PAD>'
eos_token = '<EOS>'
word_to_index[pad_token] = len(word_to_index)
word_to_index[eos_token] = len(word_to_index)
index_to_word[len(word_to_index) - 2] = pad_token
index_to_word[len(word_to_index)-1 ] = eos_token

# Step 4: Convert text sequences to numerical sequences

def text_to_sequence(text):
    return [word_to_index[word] for word in text] + [word_to_index[eos_token]]

numerical_data = [text_to_sequence(sentence) for sentence in tokenized_data]

# Step 5: Prepare the data for training

def create_batches(data, batch_size):
    random.shuffle(data)
    num_batches = len(data) // batch_size
    batches = [data[i * batch_size: (i + 1) * batch_size] for i in range(num_batches)]
    return batches

batch_size = 2
batches = create_batches(numerical_data, batch_size)

# Step 6: Define the LSTM Language Model

class LSTMLanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(LSTMLanguageModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.lstm(embedded)
        output = self.fc(output)
        return output

# Step 7: Training the LSTM Language Model

# Hyperparameters
vocab_size = len(word_to_index)
embedding_dim = 50
hidden_dim = 100
learning_rate = 0.01
num_epochs = 100

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the model and move to the device
model = LSTMLanguageModel(vocab_size, embedding_dim, hidden_dim).to(device)

# Loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    total_loss = 0
    for batch in batches:
        input_seqs = [torch.tensor(seq[:-1]).unsqueeze(0).to(device) for seq in batch]
        target_seqs = [torch.tensor(seq[1:]).unsqueeze(0).to(device) for seq in batch]

        model.zero_grad()
        loss = 0

        for i in range(batch_size):
            output = model(input_seqs[i])
            loss += loss_function(output.view(-1, vocab_size), target_seqs[i].view(-1))

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}")

# Step 8: Generate comedy routines using the trained model

def generate_comedy_routine(model, max_length=5):
    model.eval()
    start_word = random.choice(list(word_to_index.keys()))
    input_seq = torch.tensor([word_to_index[start_word]]).unsqueeze(0).to(device)
    generated_routine = []

    with torch.no_grad():
        for _ in range(max_length):
            output = model(input_seq)
            _, predicted_index = torch.max(output[:, -1, :], dim=1)
            predicted_word = index_to_word[predicted_index.item()]
            if predicted_word == eos_token:
                break
            generated_routine.append(predicted_word)
            input_seq = torch.cat((input_seq, predicted_index.unsqueeze(0)), dim=1)

    return ' '.join(generated_routine)

# Generate comedy routines
for _ in range(5):
    routine = generate_comedy_routine(model, max_length=5)
    print(routine.capitalize() + '!')

Epoch [10/100], Loss: 9.8841
Epoch [20/100], Loss: 0.5386
Epoch [30/100], Loss: 0.1732
Epoch [40/100], Loss: 0.1101
Epoch [50/100], Loss: 0.0812
Epoch [60/100], Loss: 0.0639
Epoch [70/100], Loss: 0.0524
Epoch [80/100], Loss: 0.0443
Epoch [90/100], Loss: 0.0384
Epoch [100/100], Loss: 0.0339
Laugh at me. that’s how!
Sex offenders live around you?!
Lot of bad jobs. hell,!
Like, i know, it sounds!
Guys have any kids? how!
